In [1]:
pwd

'/home/djt/Documents/Personal/end_to_end_mlops_with_python/research'

In [2]:
import os

In [4]:
%pwd

'/home/djt/Documents/Personal/end_to_end_mlops_with_python/research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'/home/djt/Documents/Personal/end_to_end_mlops_with_python'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [8]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories



In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schemas = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config
    
    

In [1]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size



In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info:\n{headers}")
        else:
            logger.info(f"File already exists of the size:{get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_files(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_files()
except Exception as e:
    raise e

[2023-10-03 15:32:09,894: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-03 15:32:09,896: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-03 15:32:09,896: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-03 15:32:09,897: INFO: common: created directory at: artifacts]
[2023-10-03 15:32:09,898: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-03 15:32:12,543: INFO: 669370419: artifacts/data_ingestion/data.zip downloaded! with following info:
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E18C:269BA:4B790:7F15D:651BE6A0
Accept-Ranges: bytes
Date: Tue, 03 Oc